In [6]:
import os
import NLPSelfSupervisedFinalProj.ner_training as ner_training
import random
from spacy.training.example import Example
import json

# Initialize spacy model
nlp = ner_training.blank("en")

# Add NER component to the pipeline
ner = nlp.add_pipe("ner")

# Process JSON data from all files in the folder
folder_path = "./LabeledDocuments"
train_data = []
i = 0
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        with open(os.path.join(folder_path, filename), "r", encoding='utf-8') as file:
            data = json.load(file)
            for item in data:
                text = item['text']
                entities = item['entities']
                annotated_entities = [(entity['start_idx'], entity['end_idx'], entity['type']) for entity in entities]
                train_data.append((text, {'entities': annotated_entities}))
            print(i)
            i += 1
    if i == 5:
        break

print("read")

# Add labels to the NER model
for _, annotations in train_data:
    for ent in annotations['entities']:
        ner.add_label(ent[2])

# Disable other pipeline components for training efficiency
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Train NER model
with nlp.disable_pipes(*unaffected_pipes):
    optimizer = nlp.begin_training()
    for itn in range(50):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in train_data:
            try:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], drop=0.5, losses=losses)
            except ValueError as e:
                # print(f"Skipping due to error: {e}")
                continue
        print(losses)

# Save model to disk
nlp.to_disk("ner_model")

ModuleNotFoundError: No module named 'spacy'